In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

## 0. Getting Started

Load the MNIST Datasets

In [2]:
feature_reduction = False
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test).to(torch.float32), torch.tensor(X0_test).to(torch.float32)
X1_test, X0_test = X1_test.permute(0, 3, 1, 2), X0_test.permute(0, 3, 1, 2)

/var/folders/ym/ry24dhs911s7zbqfbhs5ns2m0000gn/T/ipykernel_2239/314870376.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/torch/csrc/utils/tensor_new.cpp:233.)
  X1_test, X0_test = torch.tensor(X1_test).to(torch.float32), torch.tensor(X0_test).to(torch.float32)


Distance Measuring Model

In [3]:
dev = qml.device('default.qubit', wires=8)

@qml.qnode(dev, interface="torch")
def distance_circuit3(inputs): 
    embedding.QuantumEmbedding2(inputs[0:16])
    return qml.density_matrix(wires=range(8))

class Distance(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer3_distance = qml.qnn.TorchLayer(distance_circuit3, weight_shapes={})
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 16
        self.fc = torch.nn.Linear(7 * 7, 16, bias=True)
    
    def forward(self, x1, x0, measure):
        x1 = self.layer1(x1)
        x1 = self.layer2(x1)
        x1 = x1.view(-1, 7 * 7)
        x1 = self.fc(x1)

        x0 = self.layer1(x0)
        x0 = self.layer2(x0)
        x0 = x0.view(-1, 7 * 7)
        x0 = self.fc(x0)

        rhos1 = self.qlayer3_distance(x1)
        rhos0 = self.qlayer3_distance(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0

        if measure == "Trace":
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))
        elif measure == "Hilbert-Schmidt":
            return 0.5 * torch.real(torch.trace(rho_diff @ rho_diff))

## 1. Distances After the Training

Model3 Fidelity

In [4]:
Model3_Fidelity_PATH = []
for i in range(5):
    Model3_Fidelity_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model3 Fidelity/Model3_Fidelity.pt")

Model3_Fidelity_Trace_Distances, Model3_Fidelity_HS_Distances = np.array([]), np.array([])
for path in Model3_Fidelity_PATH:
    Model = Distance()
    Model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = Model(X1_test, X0_test, 'Trace')
        HS_distance = Model(X1_test, X0_test, 'Hilbert-Schmidt')
    Model3_Fidelity_Trace_Distances = np.append(Model3_Fidelity_Trace_Distances, trace_distance)
    Model3_Fidelity_HS_Distances = np.append(Model3_Fidelity_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model3_Fidelity: {Model3_Fidelity_Trace_Distances.mean()} ± {Model3_Fidelity_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model3_Fidelity: {Model3_Fidelity_HS_Distances.mean()} ± {Model3_Fidelity_HS_Distances.std()}")


/Users/tak/miniforge3/envs/QC/lib/python3.10/site-packages/pennylane/qnn/torch.py:328: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/aten/src/ATen/native/Copy.cpp:250.)
  return self.qnode(**kwargs).type(x.dtype)


Trace Distance After training with Model3_Fidelity: 0.9386046648025512 ± 0.006687183471074778
Hilbert Schmidt distance After training with Model3_Fidelity: 0.41108718514442444 ± 0.010013794751865836


Model3 HSinner

In [5]:
Model3_HSinner_PATH = []
for i in range(5):
    Model3_HSinner_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model3 HSinner/Model3_HSinner.pt")

Model3_HSinner_Trace_Distances, Model3_HSinner_HS_Distances = np.array([]), np.array([])
for path in Model3_HSinner_PATH:
    Model = Distance()
    Model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = Model(X1_test, X0_test, 'Trace')
        HS_distance = Model(X1_test, X0_test, 'Hilbert-Schmidt')
    Model3_HSinner_Trace_Distances = np.append(Model3_HSinner_Trace_Distances, trace_distance)
    Model3_HSinner_HS_Distances = np.append(Model3_HSinner_HS_Distances, HS_distance)

print(f"Trace Distance After training with Model3_HSinner: {Model3_HSinner_Trace_Distances.mean()} ± {Model3_HSinner_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model3_HSinner: {Model3_HSinner_HS_Distances.mean()} ± {Model3_HSinner_HS_Distances.std()}")

Trace Distance After training with Model3_HSinner: 0.9040618777275086 ± 0.054548215464094794
Hilbert Schmidt distance After training with Model3_HSinner: 0.3668473720550537 ± 0.046893209401121225
